In [1]:
#Book 2 Adi Shankracharya

import csv
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Load pre-trained tokenizer and model for NER
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-ner-english")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/roberta-large-ner-english")

# Define your data
mydata = "txt/Sankara-Digvijaya_images.txt"

# Initialize NER pipeline with modified aggregation_strategy
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="average")

# Chunk size for iterating over the text
chunk_size = 1000

# Initialize an empty list to store unique locations
unique_locations = []

# Write the locations to a CSV file
with open("locations.csv", "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ['word', 'start', 'end']  # Define the field names for the CSV file
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Read text from file and process in chunks
    with open(mydata, "r", encoding="utf-8") as file:
        text = file.read()

        # Preprocess the text
        text = text.replace('\n', ' ').replace('.', ' ').replace(',', ' ')

        # Process text in chunks
        for i in range(0, len(text), chunk_size):
            chunk = text[i:i+chunk_size]
            # Perform NER on the chunk
            output = nlp(chunk)
            # Write the locations to the CSV file
            for entity in output:
                if entity['entity_group'] == 'LOC':  # Check if the entity is a location
                    location = entity['word'].strip()
                    if location.isalpha() and location.istitle() and len(location) > 2:  # Filter by capitalization, length, and alphabetic characters
                        writer.writerow({'word': location, 'start': entity['start'], 'end': entity['end']})
                        unique_locations.append(location)  # Add location to the list

# Convert the list of unique locations to a DataFrame
df = pd.DataFrame({'A': unique_locations})

# Get unique values of 'A' column (locations)
unique_values = df['A'].unique()

print(unique_values)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/pipelines/token_classification.py:392: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


['India' 'Kaladi' 'Kerala' 'Samsara' 'Kaveri' 'Varanasi' 'Ganga' 'Badarf'
 'Kasi' 'Advaita' 'Horizon' 'Badari' 'Sankara' 'Iswara' 'Siva' 'Mandara'
 'Rameswaram' 'Hithalaya' 'West' 'Tamala' 'Naraka' 'Manikarnika' 'Prayaga'
 'Bharata' 'Vindhyas' 'Yamuna' 'Triveni' 'Mahishmati' 'Kashmir' 'Vidarbha'
 'Bijlabindu' 'Hastinapura' 'Raghuvamsa' 'Narmada' 'Sisupala' 'Dethi'
 'Ahavaniya' 'Anantanandagiri' 'Bombay' 'Hampi' 'Tungabhadra' 'Kanchi'
 'Madras' 'Bangalore' 'Sringeri' 'Tretayuga' 'Avidya' 'Vidya' 'Kalpaka'
 'Bapatla' 'Guntur' 'Amaruka' 'Brahmarandhra' 'Brahmaloka' 'Maharashtra'
 'Srisaila' 'Kapila' 'Kailasa' 'Samadhi' 'Ashrama' 'Bala' 'Gokarna' 'Bali'
 'Yajna' 'Mookambika' 'Sushumna' 'Manipura' 'Indra' 'Brahma' 'Vaikuntha'
 'Kalahasti' 'Suvarnamukhi' 'Pundarikapura' 'Sahya' 'Chidambaram'
 'Kanchipuram' 'Kalpataru' 'Sethu' 'South' 'Lanka' 'Asoka' 'Ravana' 'Setu'
 'Mahasurapura' 'Karnataka' 'Pandya' 'Hasti' 'Andhra' 'Tamraparni'
 'Pataliputra' 'Kapalikas' 'Dwaraka' 'Ujjayini' 'Saurashtra' 

In [ ]:
#FINDING LOCATIONS

import csv
import requests

# Replace this with the path to your input CSV file
input_csv_path = 'secondbook.csv'
# Output CSV file path
output_csv_path = 'wikidata_locations2_with_geo.csv'

def get_geolocation(place_name):
    # Wikidata endpoint for SPARQL queries
    url = 'https://query.wikidata.org/sparql'
    # SPARQL query to get geolocation for a place name
    query = f"""
    SELECT ?place ?placeLabel ?location WHERE {{
      ?place ?label "{place_name}"@en.
      ?place wdt:P625 ?location.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    LIMIT 1
    """
    headers = {'User-Agent': 'MyApp/1.0 (myemail@example.com)', 'Accept': 'application/json'}
    try:
        response = requests.get(url, headers=headers, params={'query': query, 'format': 'json'})
        response.raise_for_status()  # Raises HTTPError, if one occurred
        data = response.json()
        results = data['results']['bindings']
        if results:
            location = results[0]['location']['value']
            # Extracting latitude and longitude from the point string
            lon, lat = location.replace('Point(', '').replace(')', '').split(' ')
            return lat, lon
    except Exception as e:
        print(f"Error fetching geolocation for {place_name}: {e}")
    return None, None

# Read place names from CSV
places = []
with open(input_csv_path, mode='r', encoding='utf-8') as infile:
    reader = csv.reader(infile)
    next(reader)  # Skip header
    for row in reader:
        places.append(row[0])  # Assuming place name is in the first column

# Fetch geolocations and write to a new CSV
with open(output_csv_path, mode='w', encoding='utf-8', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['Place Name', 'Latitude', 'Longitude'])
    for place in places:
        lat, lon = get_geolocation(place)
        if lat and lon:
            writer.writerow([place, lat, lon])
            print(f"Geolocation for {place}: {lat}, {lon}")
        else:
            print(f"No geolocation found for {place}")

print("\nDone writing geolocations to CSV.")

In [ ]:
import csv

# Given list of codes
locations = ['India', 'Kaladi', 'Kerala', 'Kaveri', 'Varanasi', 'Kasi', 'Advaita', 'Horizon', 'Badarinath', 'Mandara', 'Rameswaram', 'Hithalaya', 'Tamala', 'Naraka', 'Manikarnika', 'Prayaga', 'Bharata', 'Vindhyas', 'Triveni', 'Mahishmati', 'Kashmir', 'Vidarbha', 'Bijlabindu', 'Hastinapura', 'Raghuvamsa', 'Narmada', 'Sisupala', 'Dethi', 'Ahavaniya', 'Anantanandagiri', 'Bombay', 'Hampi', 'Tungabhadra', 'Kanchi', 'Madras', 'Bangalore', 'Sringeri', 'Tretayuga', 'Avidya', 'Vidya', 'Kalpaka', 'Bapatla', 'Guntur', 'Amaruka', 'Brahmarandhra', 'Brahmaloka', 'Maharashtra', 'Srisaila', 'Kapila', 'Kailasa', 'Ashrama', 'Bala', 'Gokarna', 'Bali', 'Yajna', 'Mookambika', 'Manipura', 'Kalahasti', 'Suvarnamukhi', 'Pundarikapura', 'Sahya', 'Chidambaram', 'Kanchipuram', 'Kalpataru', 'Sethu', 'Lanka', 'Mahasurapura', 'Karnataka', 'Pandya', 'Hasti', 'Andhra', 'Tamraparni', 'Pataliputra', 'Kapalikas', 'Dwaraka', 'Ujjayini', 'Saurashtra', 'Kamakoti', 'Nidhi', 'Bahlika', 'Surasena', 'Darada', 'Kuru', 'Panchala', 'Kamarupa', 'Kosala', 'Anga', 'Gauda', 'Kanada', 'Kshapanakas', 'Mysore', 'Gaddi', 'Ujjain', 'Badami', 'Mandukya', 'Saradapitha', 'Kedara', 'Sringagiri', 'Selukos', 'Syria', 'Egypt', 'Macedonia', 'Cyrene', 'Epirus', 'Dwarka', 'Totaka', 'Puri', 'Padmapada', 'Chennai', 'Mylapore', 'Puris', 'Sringen', 'Jyothi', 'Kedar', 'Kedarnath', 'Badrinath', 'Samadhisthan', 'Kailash', 'Alathur', 'Calcutta', 'Trichur', 'Vrishachala', 'Badarinarayan', 'Marathawada', 'Mahuripuri', 'Mahur', 'Mahuragad', 'Yavatmal', 'Sasalagrama', 'Sasilagrama', 'Kaaladi', 'Kal', 'Palikode', 'Trippunithura', 'Ponnur', 'London', 'Ullur', 'Trivandrum', 'Devaloka', 'Sudhanvan', 'Jaina', 'Sivalingam', 'Purna', 'Vadakkunathan', 'Periyar', 'Gurukula', 'Garhyapatyagni', 'Ahavaniyagni', 'Lagna', 'Meru', 'Moksha', 'Sona', 'Kundina', 'Gokula', 'Vyaghrapura', 'Dwaita', 'Tilaka', 'Munja', 'Kashaya', 'Sruti', 'Tripurds', 'Kutaja', 'Manasa', 'Reva', 'Himalayas']

# Define the file name
file_name = 'place_and_locations2.csv'

# Write the codes to a CSV file
with open(file_name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for location in locations:
        writer.writerow([location])

print(f"CSV file '{file_name}' has been created successfully.")

In [ ]:
import folium

locations = {
    "Kaladi": (10.819338, 76.002973),
    "Kerala": (10.0, 76.3),
    "Kaveri": (11.370345, 79.828105),
    "Varanasi": (25.318888888, 83.012777777),
    "Mandara": (22.9087, 88.11603),
    "Rameswaram": (9.28, 79.3),
    "Prayaga": (25.45, 81.85),
    "Bharata": (22.8, 83.0),
    "Mahishmati": (22.176944444, 75.585555555),
    "Kashmir": (34.5, 76.0),
    "Vidarbha": (21.15, 79.09),
    "Bombay": (19.075833333, 72.8775),
    "Hampi": (15.334547, 76.462162),
    "Madras": (13.0825, 80.275),
    "Bangalore": (12.9791198, 77.5912997),
    "Sringeri": (13.42, 75.26),
    "Bapatla": (15.905, 80.468),
    "Guntur": (16.3, 80.45),
    "Maharashtra": (18.97, 72.82),
    "Gokarna": (14.55, 74.31667),
    "Manipura": (13.419166666, 75.149444444),
    "Kalahasti": (13.75, 79.7),
    "Chidambaram": (11.399444444, 79.693611111),
    "Kanchipuram": (12.830833333, 79.707777777),
    "Karnataka": (15.0, 76.0),
    "Hasti": (30.635555555, 79.755),
    "Andhra": (16.52, 79.5),
    "Pataliputra": (25.61, 85.141388888),
    "Dwaraka": (22.240277777, 68.968611111),
    "Ujjayini": (23.182777777, 75.777222222),
    "Saurashtra": (22.3, 70.7833),
    "Mysore": (12.308611111, 76.653055555),
    "Ujjain": (23.182777777, 75.777222222),
    "Badami": (15.91495, 75.67683),
    "Dwarka": (22.240277777, 68.968611111),
    "Puri": (19.8, 85.816666666),
    "Chennai": (13.0825, 80.275),
    "Mylapore": (13.0336, 80.2687),
    "Kedarnath": (30.73333333, 79.06666667),
    "Badrinath": (30.742425, 79.496038888),  # Corrected duplicate key
    "Kailash": (31.066944444, 81.312777777),
    "Alathur": (10.648055555, 76.538333333),
    "Calcutta": (22.5726723, 88.3638815),
    "Yavatmal": (20.4, 78.133333333),
    "Ponnur": (16.0667, 80.5667),
    "Ullur": (10.975, 79.401388888),
    "Trivandrum": (8.4875, 76.9525),
    "Purna": (21.0958, 76.01),
    "Kundina": (20.55, 77.45),
    "Manasa": (24.48, 75.15),
    "Himalayas": (29.0, 84.0)
}
# Create a map centered around the midpoint of the journey
map = folium.Map(location=[38.6270, -90.1994], zoom_start=5)

# Adding markers to the map
for city, coords in locations.items():
    folium.Marker(
        location=coords,
        popup=city,
        icon=folium.Icon(color='blue')  # Example of using a blue marker
    ).add_to(map)

# Extracting the coordinates from the locations dictionary
polyline_coords = list(locations.values())

# Add lines between the locations
folium.PolyLine(
    locations=polyline_coords,
    color="blue"
).add_to(map)

# Display the map
map.save('travel_heatmap_book2.html')